# Practice training a deep neural network on the CIFAR10 image dataset

In [13]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

import os

### Build a DNN with 20 hidden layers of 100 neurons each (that’s too many, but it’s the point of this exercise). Use He initialization and the ELU activation function.

Clearing the session removes all the nodes left over from previous models, freeing memory and preventing slowdown.

In [4]:
keras.backend.clear_session()

tf.random.set_seed(42)
np.random.seed(42)

In [5]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal"))

### Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with keras.datasets.cifar10.load_data(). The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you'll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model's architecture or hyperparameters.

#### Adding the output layer to the model:

In [6]:
model.add(keras.layers.Dense(10, activation="softmax"))

Nadam optimizer with a learning rate of 5e-5

In [7]:
optimizer = keras.optimizers.Nadam(lr=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

#### Loading CIFAR10 dataset. We also want to use early stopping, so we need a validation set. Let's use the first 5,000 images of the original training set as the validation set:

In [9]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

In [10]:
X_train_full.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [11]:
X_test

array([[[[158, 112,  49],
         [159, 111,  47],
         [165, 116,  51],
         ...,
         [137,  95,  36],
         [126,  91,  36],
         [116,  85,  33]],

        [[152, 112,  51],
         [151, 110,  40],
         [159, 114,  45],
         ...,
         [136,  95,  31],
         [125,  91,  32],
         [119,  88,  34]],

        [[151, 110,  47],
         [151, 109,  33],
         [158, 111,  36],
         ...,
         [139,  98,  34],
         [130,  95,  34],
         [120,  89,  33]],

        ...,

        [[ 68, 124, 177],
         [ 42, 100, 148],
         [ 31,  88, 137],
         ...,
         [ 38,  97, 146],
         [ 13,  64, 108],
         [ 40,  85, 127]],

        [[ 61, 116, 168],
         [ 49, 102, 148],
         [ 35,  85, 132],
         ...,
         [ 26,  82, 130],
         [ 29,  82, 126],
         [ 20,  64, 107]],

        [[ 54, 107, 160],
         [ 56, 105, 149],
         [ 45,  89, 132],
         ...,
         [ 24,  77, 124],
        

In [12]:
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [25]:
X_train.shape

(45000, 32, 32, 3)

#### Creating the callbacks and training the model

In [14]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)

run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

#### %tensorboard --logdir=./my_cifar10_logs --port=6006

In [16]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=callbacks)

Epoch 1/100
   1/1407 [..............................] - ETA: 0s - loss: 165.9660 - accuracy: 0.0625WARNING:tensorflow:From /Users/paulmospan/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1407/1407 [==============================] - 10s 7ms/step - loss: 4.1571 - accuracy: 0.1532 - val_loss: 2.1801 - val_accuracy: 0.2046
Epoch 2/100
1407/1407 [==============================] - 9s 6ms/step - loss: 2.0763 - accuracy: 0.2407 - val_loss: 2.0508 - val_accuracy: 0.2356
Epoch 3/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.9539 - accuracy: 0.2802 - val_loss: 1.9222 - val_accuracy: 0.2888
Epoch 4/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.8691 - accuracy: 0.3164 - val_loss: 1.9374 - val_accuracy: 0.3124
Epoch 5/100
1407/1407 [=========

In [17]:
model = keras.models.load_model("my_cifar10_model.h5")

In [18]:
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.5252 - accuracy: 0.0942


[1.5251532793045044, 0.094200000166893]

###  Adding Batch Normalization and compare the learning curves: Is it converging faster than before? Does it produce a better model? How does it affect training speed?

BN layer added after every Dense layer (before the activation function), except for the output layer. Plus one BN layer before the first hidden layer.

Learning rate changed to 5e-4.

Run directories renamed to runbn* and the model file name to my_cifar10_bn_model.h5

In [19]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [20]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())

for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(lr=5e-4)

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 3072)              12288     
_________________________________________________________________
dense (Dense)                (None, 100)               307300    
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               400       
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               4

In [22]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [23]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [24]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.8408 - accuracy: 0.3389 - val_loss: 1.7057 - val_accuracy: 0.3910
Epoch 2/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.6651 - accuracy: 0.4066 - val_loss: 1.5858 - val_accuracy: 0.4318
Epoch 3/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5956 - accuracy: 0.4340 - val_loss: 1.5351 - val_accuracy: 0.4366
Epoch 4/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5451 - accuracy: 0.4498 - val_loss: 1.5087 - val_accuracy: 0.4670
Epoch 5/100
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5044 - accuracy: 0.4649 - val_loss: 1.4338 - val_accuracy: 0.4872
Epoch 6/100
1407/1407 [==============================] - 13s 10ms/step - loss: 1.4677 - accuracy: 0.4774 - val_loss: 1.4202 - val_accuracy: 0.4940
Epoch 7/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.4352 - accuracy: 0.4882 - val_loss: 1.

In [26]:
model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.3256 - accuracy: 0.0900


[1.3256243467330933, 0.09000000357627869]

### Replacing Batch Normalization with SELU, and making the necessary adjustements to ensure the network self-normalizes (i.e., standardize the input features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers, etc.).

In [27]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [28]:
optimizer = keras.optimizers.Nadam(lr=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [29]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [30]:
X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

In [31]:
model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.9349 - accuracy: 0.3018 - val_loss: 1.8360 - val_accuracy: 0.3250
Epoch 2/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.7235 - accuracy: 0.3900 - val_loss: 1.7700 - val_accuracy: 0.3612
Epoch 3/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.6253 - accuracy: 0.4247 - val_loss: 1.7223 - val_accuracy: 0.4002
Epoch 4/100
1407/1407 [==============================] - 9s 6ms/step - loss: 1.5581 - accuracy: 0.4510 - val_loss: 1.6428 - val_accuracy: 0.4280
Epoch 5/100
1407/1407 [==============================] - 9s 7ms/step - loss: 1.5062 - accuracy: 0.4714 - val_loss: 1.6041 - val_accuracy: 0.4366
Epoch 6/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.4547 - accuracy: 0.4899 - val_loss: 1.5172 - val_accuracy: 0.4786
Epoch 7/100
1407/1407 [==============================] - 10s 7ms/step - loss: 1.4115 - accuracy: 0.5046 - val_loss: 1.5435 - val_a

In [32]:
model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.4646 - accuracy: 0.1116


[1.464646577835083, 0.11159999668598175]